# Create small netcdf
We import the arrays generated in `firesmoke_make_metadata.ipynb` as well as additional metadata from original netCDF file.

## Import necessary libraries

In [ ]:
# to open original netcdfs and new netcdf with idx url
import xarray as xr
import numpy as np
# for connecting OpenVisus framework to xarray
# from https://github.com/sci-visus/openvisuspy, 
from openvisuspy.xarray_backend import OpenVisusBackendEntrypoint

## Load the original metadata from one dispersion.nc file

Get path to a netcdf file containing metadata

In [ ]:
# ******* THIS IS WHEN RUNNING FROM ATLANTIS.SCI **************
firesmoke_file = "/usr/sci/cedmav/data/firesmoke/BSC00CA12-01/dispersion_20240103.nc"

In [ ]:
# load the original netcdf file
ds = xr.open_dataset(firesmoke_file)

# take a look
ds

Drop the TFLAGs and create a new TFLAGs variable with all 27k timesteps. 
Drop PM25 variable, this is the variable of interest and already in IDX file.
Drop the attributes that are unique to this file, we're adding variables to represent all this metadata instead.

In [ ]:
# drop the TFLAG variable and add our own, also drop PM25, this is the array stored by idx
# ref: https://docs.xarray.dev/en/stable/generated/xarray.DataArray.drop_vars.html
ds_new = ds.drop_vars("TFLAG")
ds_new = ds_new.drop_vars("PM25")

# delete the attributes we are now adding as variables
attrs = ['CDATE', 'CTIME', 'WDATE', 'WTIME', 'SDATE', 'STIME']
for attr in attrs:
    del ds_new.attrs[attr]

# ds_new is now empty except for attributes
ds_new

## Create new variables for metadata from `firesmoke_make_metadata.ipynb` and add to our netCDF file

In [ ]:
# import metadata
tflags = np.load("firesmoke_v4-tflags.npy")
init_tflags = np.load("firesmoke_v4-last_init.npy")
resamp = np.load("firesmoke_v4-resamp.npy")

attr_data = {attr: [] for attr in attrs}
for attr in attrs:
    attr_data[attr] = np.load(f"firesmoke_v4-{attr}.npy")

# reshape timeflags so they're exactly like UBC's original tflags
tflags = np.expand_dims(tflags, axis=1)
init_tflags = np.expand_dims(init_tflags, axis=1)

# make timeflags int32, as in original netcdf
tflags = tflags.astype(np.int32)
init_tflags = init_tflags.astype(np.int32)

# create dimensions for these arrays of metadata, stay as close as possible to UBC style dimenion naming convention
# ref: https://docs.xarray.dev/en/latest/generated/xarray.Dataset.assign.html
ds_new = ds_new.assign({'TFLAG':(['time', 'VAR', 'DATE-TIME'], tflags)})
ds_new = ds_new.assign({'wrf_arw_init_time':(['time', 'VAR', 'DATE-TIME'], init_tflags)})
ds_new = ds_new.assign({'resampled':(['time'], resamp)})

for attr in attrs:
    ds_new = ds_new.assign({attr:(['time'], attr_data[attr])})

# take a look, now we just need to add pm25
ds_new

In [ ]:
# Create a PM25 dataarray representing dims of one time slice
pm25_var = xr.DataArray(np.zeros((ds.NROWS, ds.NCOLS)), dims=['ROW', 'COL'], attrs=ds['PM25'].attrs)

# add PM25 to dataset, this is strictly for storing the dimensionality of PM25
# as this is dimensionality info is not available to xarray from idx file
ds_new['PM25'] = pm25_var

### Add idx_url as an attribute, then we're ready to save our tiny netcdf.

In [ ]:
# url to idx file
idx_url = "http://atlantis.sci.utah.edu/mod_visus?dataset=UBC_fire_smoke_BSC&cached=1"

# Adding a new idx_url attribute
ds_new = ds_new.assign_attrs({'idx_url': idx_url})

In [ ]:
# directory to new tiny netcfd file
new_nc = "firesmoke_metadata.nc"

# save xarray dataset as netcdf, this is the metadata we put into tiny netcdf
ds_new.to_netcdf("firesmoke_metadata.nc")

In [ ]:
# check out the tiny netcdf loaded by vanilla xarray
xr.open_dataset(new_nc)

## Test tiny netcdf

## Load tiny netcdf file with backend_v3 and pass timestamps above as timesteps

In [ ]:
# load data
final_ds = xr.open_dataset(new_nc, engine=OpenVisusBackendEntrypoint)
final_ds